Code edited from https://github.com/backyardbiomech/DLCconverterDLT/tree/master
Nishka Munpally
Feb 18 2025

In [2]:
from pathlib import Path
from deeplabcut.utils.auxiliaryfunctions import read_config
from deeplabcut.utils import conversioncode
import os
import numpy as np
import re
import pandas as pd

Loading DLC 3.0.0rc6...


In [3]:
# all our inputs
cnum = '3'
# make paths into Paths
config=Path('./Chase-Nishka-2025-02-11/config.yaml')
xyfname=Path('C:\\Users\\nm834\\Desktop\\DataAnalysisNishka\\DigitizedPoints\\DLTdv8_data_trial7_400rpm_intactxypts.csv')

vid=Path('./Chase-Nishka-2025-02-11/videos/Trial7_400rpm_20655.mp4')
camname=vid.stem

#load dlc config
cfg = read_config(config)
labdir = Path(cfg['project_path']) / 'labeled-data' / camname
scorer = cfg['scorer']
ma = cfg['multianimalproject']
if ma:
    individuals = cfg['individuals']
    indiv = individuals[ind]
    bodyparts = cfg['multianimalbodyparts']
else:
    bodyparts=cfg['bodyparts']
    coords = ['x', 'y']

# load xypts file to dataframe
xypts = pd.read_csv(xyfname)
xypts = xypts.astype('float64')
# make all columns lowercase for argus DLT compatibility
xypts.columns = [c.lower() for c in xypts.columns]
# just get the columns for this camera
camstr = 'cam_{}_'.format(cnum)
thiscam = [x for x in xypts.columns if camstr in x]
if len(thiscam ) ==0:
    #DLTdv8 naming
    camstr = 'cam{}_'.format(cnum)
    thiscam = [x for x in xypts.columns if camstr in x]
xypts = xypts[thiscam]
newcols = {}
# store track name and column index - start of tracks - in dict
for i in range(0, len(xypts.columns), 2):
    newcol = xypts.columns[i].split('_')[0]
    newcols[newcol]=i
    
# change the column name to match the DLC names
# added Tanvi Deora
# 18 Feb 2025
key = {'pt1':'bead',
       'pt2': 'head',
        'pt3':'left_wing_base',
       'pt4': 'right_wing_base',
       'pt5': 'left_wing_tip',
       'pt6': 'right_wing_tip'}

new_columns = []
for colname in xypts.columns:
    a,b,c = colname.split('_')
#     print(key[a])
    new_colname = key[a] + '_' + b + '_' + c
    new_columns.append(new_colname)
    
xypts.columns = new_columns
################################################################   

# find existing ./CollectedData_scorerintitials.h5 in labdir
colldata = list(labdir.glob('**/CollectedData_*.h5'))

# find extracted images
imgs = list(labdir.glob('**/*.png'))


if len(colldata) == 0:
        # no file exists, check to see if images extracted, and make the file based on config
        index = ['labeled-data{}{}{}{}'.format(os.sep, camname, os.sep, im.name) for im in imgs]
        # build the empty df
        # get tracknames from cfg
        if ma:
            header = pd.MultiIndex.from_product([[scorer],
                                             individuals,
                                             bodyparts,
                                             coords],
                                             names=['scorer', 'individuals','bodyparts', 'coords'])
        else:
            header = pd.MultiIndex.from_product([[scorer],
                                                 bodyparts,
                                                 coords],
                                                names=['scorer', 'bodyparts', 'coords'])
        #copy just the rows of interest from DLT to a new df
        #get frame numbers from imgs, from full path, after img, before .png
        #idx = [int(re.findall(r'\d+', s)[0]) for s in [x.stem for x in imgs]]
        df = pd.DataFrame(np.nan, columns=header, index=index)
        
conversioncode.guarantee_multiindex_rows(df)
df.sort_index(inplace=True)


# go through df find indexes without any entries, extract those entries from xydata, and add
news = df.index[df.isnull().all(1)]
# go through news and get insert digitized points from xydata
for new in news:
    for bp in bodyparts:
        xyrow = int(re.findall(r'img(\d+)\.png', new[2])[0])
        try:
            if ma:
                df.loc[new, (scorer, indiv, bp, ['x', 'y'])] = xypts.loc[
                    xyrow, ['{}_{}x'.format(bp, camstr), '{}_{}y'.format(bp, camstr)]].values
            else:
                df.loc[new, (scorer, bp, ['x', 'y'])] = xypts.loc[
                    xyrow, ['{}_{}x'.format(bp, camstr), '{}_{}y'.format(bp, camstr)]].values
        except:
            # image or xypts row not found due to offsets deletions
            continue


# replace DLT nans with empty entries for DLC formatting
df.astype('float64')
df.sort_index(inplace=True)

# # save out hdf and csv files
df.to_hdf(Path(labdir) / ('CollectedData_' + scorer + '.h5'), 'df_with_missing')#, format='table', mode='w')
df.to_csv(Path(labdir) / ('CollectedData_' + scorer + '.csv'))

C:\Users\nm834\AppData\Local\Temp\ipykernel_16404\3345925658.py:120: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  df.to_hdf(Path(labdir) / ('CollectedData_' + scorer + '.h5'), 'df_with_missing')#, format='table', mode='w')


In [4]:
df.tail()

scorer                                           Nishka              \
bodyparts                                          bead               
coords                                                x           y   
labeled-data Trial7_400rpm_20655 img906.png  891.821829  450.701118   
                                 img907.png  892.056637  450.410516   
                                 img908.png  892.056637  450.991719   
                                 img909.png  892.056637  450.991719   
                                 img910.png  891.587021  452.444726   

scorer                                                               \
bodyparts                                          head               
coords                                                x           y   
labeled-data Trial7_400rpm_20655 img906.png  872.982250  445.081809   
                                 img907.png  873.681867  444.623544   
                                 img908.png  872.982250  445.310941   
                                 img909.png  870.883397  447.373129   
                                 img910.png  869.950574  447.831393   

scorer                                                                  \
bodyparts                                   left_wing_base               
coords                                                   x           y   
labeled-data Trial7_400rpm_20655 img906.png     878.364669  464.171470   
                                 img907.png     878.364669  464.171470   
                                 img908.png     875.028812  466.257205   
                                 img909.png     875.028812  465.065356   
                                 img910.png     874.119033  464.767394   

scorer                                                                   \
bodyparts                                   right_wing_base               
coords                                                    x           y   
labeled-data Trial7_400rpm_20655 img906.png      900.542161  477.489975   
                                 img907.png      890.948454  484.297726   
                                 img908.png      877.090878  482.988543   
                                 img909.png             NaN         NaN   
                                 img910.png             NaN         NaN   

scorer                                                                 \
bodyparts                                   left_wing_tip               
coords                                                  x           y   
labeled-data Trial7_400rpm_20655 img906.png    900.230023  478.568155   
                                 img907.png    891.435792  484.066723   
                                 img908.png    876.645494  483.281213   
                                 img909.png           NaN         NaN   
                                 img910.png           NaN         NaN   

scorer                                                              
bodyparts                                   right_wing_tip          
coords                                                   x       y  
labeled-data Trial7_400rpm_20655 img906.png         795.07  305.94  
                                 img907.png         799.65  305.81  
                                 img908.png         804.58  305.75  
                                 img909.png         809.41  305.73  
                                 img910.png         814.07  305.69